# 환경 설정

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.5 MB/s eta 0:00:00


# 데이터 압축 해제

In [3]:
# zip 파일 읽어서 압축 해제

import os
import zipfile

# 경로 설정
base_dir = "/content/YOLO_safety"   # colab 로컬 디렉토리
os.makedirs(base_dir, exist_ok=True)

zip_path = "/content/drive/MyDrive/AI/"

# 압축 해제 함수
def unzip_files(zip_path, target_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"압축 해제 완료: {zip_path} -> {target_dir}")

# 압축 해제
file_name = "1_completed.zip"
unzip_files(zip_path + file_name, base_dir)

# 압축 해제
file_name = "2_completed.zip"
unzip_files(zip_path + file_name, base_dir)

압축 해제 완료: /content/drive/MyDrive/AI/1_completed.zip -> /content/YOLO_safety
압축 해제 완료: /content/drive/MyDrive/AI/2_completed.zip -> /content/YOLO_safety


# JSON 라벨을 YOLO 포맷으로 변환

In [4]:
import json
from pathlib import Path

def convert_box_json_to_yolo(json_dir, output_dir, class_map):
    """
    box 또는 polygon 기반 JSON 라벨을 YOLO 포맷으로 변환하는 함수.
    polygon은 좌표의 min/max로 bounding box를 추정.
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    json_files = list(json_dir.glob("*.json"))
    print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.")

    converted_count = 0  # 실제 변환된 JSON 수

    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        filename = data["image"]["filename"]
        img_width, img_height = data["image"]["resolution"]
        label_lines = []

        for ann in data.get("annotations", []):
            class_id_raw = ann.get("class")
            if class_id_raw not in class_map:
                continue

            class_id = class_map[class_id_raw]

            # 1. bbox가 있으면 우선 사용
            box = ann.get("box")
            if box and len(box) == 4:
                xmin, ymin, xmax, ymax = box
            # 2. polygon만 있는 경우
            elif "polygon" in ann and len(ann["polygon"]) >= 3:
                xs = [pt[0] for pt in ann["polygon"]]
                ys = [pt[1] for pt in ann["polygon"]]
                xmin, xmax = min(xs), max(xs)
                ymin, ymax = min(ys), max(ys)
            else:
                continue    # 유효한 box도 polygon도 없는 경우

            cx = ((xmin + xmax) / 2) / img_width
            cy = ((ymin + ymax) / 2) / img_height
            w = (xmax - xmin) / img_width
            h = (ymax - ymin) / img_height

            label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

        if label_lines:
            label_file = output_dir / (Path(filename).stem + ".txt")
            with open(label_file, "w") as out_f:
                out_f.write("\n".join(label_lines))
            converted_count += 1  # ✅ 변환된 경우만 카운트 증가

    print(f"✅ YOLO 라벨로 변환 완료: {converted_count}개 파일 생성됨")

In [5]:
# 클래스 매핑: JSON 클래스 번호(str) → YOLO 클래스 인덱스(int)
class_mapping_all = {
    "01": 0,  # 안전벨트 착용
    "03": 1,  # 안전고리 결착
    "07": 2,  # 안전모 착용
}

# ✅ 클래스별로 반복 처리
for cls_str, yolo_idx in class_mapping_all.items():
    # 학습 데이터
    convert_box_json_to_yolo(
        json_dir=f"/content/YOLO_safety/train/labels_json/{cls_str}",
        output_dir=f"/content/YOLO_safety/train/labels",
        class_map={cls_str: yolo_idx}  # 해당 클래스만 변환
    )
    print(f"클래스: {cls_str} 학습 데이터 라벨 변환 완료")

    # 검증 데이터
    convert_box_json_to_yolo(
        json_dir=f"/content/YOLO_safety/val/labels_json/{cls_str}",
        output_dir=f"/content/YOLO_safety/val/labels",
        class_map={cls_str: yolo_idx}  # 해당 클래스만 변환
    )
    print(f"클래스: {cls_str} 검증 데이터 라벨 변환 완료\n")

총 815개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 794개 파일 생성됨
클래스: 01 학습 데이터 라벨 변환 완료
총 84개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 84개 파일 생성됨
클래스: 01 검증 데이터 라벨 변환 완료

총 414개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 414개 파일 생성됨
클래스: 03 학습 데이터 라벨 변환 완료
총 77개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 77개 파일 생성됨
클래스: 03 검증 데이터 라벨 변환 완료

총 802개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 802개 파일 생성됨
클래스: 07 학습 데이터 라벨 변환 완료
총 87개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 87개 파일 생성됨
클래스: 07 검증 데이터 라벨 변환 완료



In [6]:
from pathlib import Path

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

train_count = len(list(train_label_dir.glob("*.txt")))
val_count = len(list(val_label_dir.glob("*.txt")))

print(f"✅ 총 라벨 파일 개수: {train_count + val_count}개")
print(f" - 학습: {train_count}개")
print(f" - 검증: {val_count}개")

✅ 총 라벨 파일 개수: 2227개
 - 학습: 2002개
 - 검증: 225개


# 이미지 경로 평탄화

In [8]:
import shutil
from pathlib import Path

def flatten_image_dir_skip_duplicates(img_base_dir: Path):
    flat_img_dir = img_base_dir

    for subdir in img_base_dir.iterdir():
        if subdir.is_dir():
            for img_file in subdir.glob("*.jpg"):
                target_path = flat_img_dir / img_file.name

                if target_path.exists():
                    print(f"⚠️ 중복으로 건너뜀: {img_file.name}")
                    continue

                shutil.move(str(img_file), str(target_path))

    print(f"✅ 중복 건너뛰며 평탄화 완료: {img_base_dir}")

# 적용
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/train/images"))
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/val/images"))

⚠️ 중복으로 건너뜀: S2-O1301M03390.jpg
⚠️ 중복으로 건너뜀: S2-N0108M17282.jpg
⚠️ 중복으로 건너뜀: S2-N0108M17281.jpg
⚠️ 중복으로 건너뜀: S2-N1001M00214.jpg
⚠️ 중복으로 건너뜀: S2-N1001M00216.jpg
⚠️ 중복으로 건너뜀: S2-N1001M00211.jpg
⚠️ 중복으로 건너뜀: S2-N0108M17283.jpg
⚠️ 중복으로 건너뜀: S2-N1001M00202.jpg
✅ 중복 건너뛰며 평탄화 완료: /content/YOLO_safety/train/images
⚠️ 중복으로 건너뜀: S2-O1301M04136.jpg
⚠️ 중복으로 건너뜀: S2-N0108M07081.jpg
⚠️ 중복으로 건너뜀: S2-N0109M03154.jpg
⚠️ 중복으로 건너뜀: S2-O1301M03994.jpg
⚠️ 중복으로 건너뜀: S2-N0109M16109.jpg
⚠️ 중복으로 건너뜀: S2-N1206M03695.jpg
⚠️ 중복으로 건너뜀: S2-O1301M04115.jpg
⚠️ 중복으로 건너뜀: S2-N0109M16730.jpg
⚠️ 중복으로 건너뜀: S2-N0108M15892.jpg
⚠️ 중복으로 건너뜀: S2-N0109M07066.jpg
⚠️ 중복으로 건너뜀: S2-N0109M07930.jpg
⚠️ 중복으로 건너뜀: S2-N0109M06451.jpg
⚠️ 중복으로 건너뜀: S2-O1301M02608.jpg
⚠️ 중복으로 건너뜀: S2-N0109M00577.jpg
⚠️ 중복으로 건너뜀: S2-N0108M20782.jpg
⚠️ 중복으로 건너뜀: S2-N1206M01586.jpg
⚠️ 중복으로 건너뜀: S2-N0109M07797.jpg
⚠️ 중복으로 건너뜀: S2-N0109M03052.jpg
⚠️ 중복으로 건너뜀: S2-O1301M04195.jpg
⚠️ 중복으로 건너뜀: S2-N0108M07190.jpg
⚠️ 중복으로 건너뜀: S2-O1301M03900.jpg
⚠️ 중복으로 건너뜀: S2-N010

# 데이터 존재 확인

In [9]:
from pathlib import Path

# 이미지 디렉토리 존재 확인
print("✅ train images exists:", Path("/content/YOLO_safety/train/images").exists())
print("✅ val images exists:", Path("/content/YOLO_safety/val/images").exists())

# 라벨 디렉토리 + 파일 개수 확인
val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
print(f"✅ val 라벨 수: {len(val_labels)}개")

# 비어있는 파일 확인
empty = [f for f in val_labels if f.stat().st_size == 0]
print(f"⚠️ 비어있는 라벨 파일 수: {len(empty)}개")

✅ train images exists: True
✅ val images exists: True
✅ val 라벨 수: 225개
⚠️ 비어있는 라벨 파일 수: 0개


In [10]:
# 잘못된 클래스 ID가 있는지 확인
bad_lines = 0
for f in val_labels:
    lines = f.read_text().splitlines()
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        try:
            class_id = int(parts[0])
            if class_id not in [0, 1, 2]:
                print(f"⚠️ 잘못된 클래스 ID: {class_id} in {f.name}")
                bad_lines += 1
        except:
            print(f"⚠️ 형식 오류 in {f.name}: {line}")
            bad_lines += 1

print(f"❗ 잘못된 라벨 줄 수: {bad_lines}")

❗ 잘못된 라벨 줄 수: 0


In [11]:
# 라벨 파일 몇 개 출력해보기 (예: 처음 3개)
for f in val_labels[:3]:
    print(f"\n📄 파일명: {f.name}")
    print(f.read_text())


📄 파일명: S2-N0108M17275.txt
1 0.671354 0.478704 0.151042 0.085185

📄 파일명: S2-N1106M19759.txt
1 0.563802 0.390278 0.075521 0.099074

📄 파일명: S2-N1106M20523.txt
1 0.586198 0.254167 0.120313 0.376852


In [12]:
val_img_dir = Path("/content/YOLO_safety/val/images")
val_images = [f.stem for f in val_img_dir.glob("*.jpg")]
val_labels = [f.stem for f in Path("/content/YOLO_safety/val/labels").glob("*.txt")]

no_label_images = set(val_images) - set(val_labels)

print(f"⚠️ 라벨이 없는 이미지 수: {len(no_label_images)}")
print(list(no_label_images)[:5])  # 일부 예시

⚠️ 라벨이 없는 이미지 수: 0
[]


In [13]:
import os

# 존재 확인
print("val 이미지 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/images"))
print("val 라벨 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/labels"))

val 이미지 디렉토리 있음: True
val 라벨 디렉토리 있음: True


# 학습

In [14]:
yaml_text = """
path: /content/YOLO_safety
train: train/images
val: val/images

names:
  0: safety_harness
  1: safety_lanyard
  2: safety_helmet
"""

with open("/content/YOLO_safety/safety.yaml", "w") as f:
    f.write(yaml_text)

In [15]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt -> yolov8n.pt -> yolov8m.pt

model.train(
    data="/content/YOLO_safety/safety.yaml",
    epochs=50,
    imgsz=640,
    project="/content/drive/MyDrive/YOLO_safety/runs",  # 저장 경로
    name="safetyGear_detect_exp1"                       # 실험명
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 343MB/s]

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/YOLO_safety/safety.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=safetyGear_detect_exp13, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/content/drive/MyDrive/YOLO_saf

100%|██████████| 755k/755k [00:00<00:00, 161MB/s]

...

                   all        248        336      0.488      0.584      0.423      0.283



50 epochs completed in 0.826 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp13/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp13/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp13/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.02it/s]


                   all        248        336       0.53      0.612      0.464       0.29
        safety_harness         61         97      0.291      0.732      0.368      0.204
        safety_lanyard         77         88      0.814      0.398      0.487      0.294
         safety_helmet         87        151      0.485      0.705      0.538      0.373
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp13


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d8df0a4ee10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [17]:
import os
from collections import Counter

def count_yolo_classes(label_dir, class_names=None):
    """
    YOLO 라벨(.txt) 파일에서 클래스별 인스턴스 수를 카운트합니다.

    :param label_dir: 라벨 파일(.txt)들이 있는 디렉토리
    :param class_names: 클래스 이름 리스트 (있으면 이름과 함께 출력됨)
    """
    counter = Counter()

    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r", encoding="utf-8") as f:
                for line in f:
                    class_id = line.strip().split()[0]  # 첫 번째 값이 class_id
                    counter[int(class_id)] += 1

    print("📊 클래스별 인스턴스 수:")
    for class_id, count in sorted(counter.items()):
        name = class_names[class_id] if class_names else f"class_{class_id}"
        print(f"  {name} ({class_id}): {count}개")

    return counter

label_dir = "/content/YOLO_safety/train/labels"  # 또는 val
class_names = ["safety_harness", "safety_lanyard", "safety_helmet"]
count_yolo_classes(label_dir, class_names)

📊 클래스별 인스턴스 수:
  safety_harness (0): 1240개
  safety_lanyard (1): 438개
  safety_helmet (2): 1367개


Counter({2: 1367, 0: 1240, 1: 438})